In [2]:
import numpy as np
import pandas as pd

In [3]:
# Reading the dataset using Pandas
df = pd.read_csv("LBW_Dataset.csv")

# Normalization & Scaling Functions using Numpy & Pandas

# Outlier Scaling using .quantile() Pandas methods
def scale_outlier(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    min_bound = Q1 - 1.5*IQR
    max_bound = Q3 + 1.5*IQR
    df[column] = np.where(df[column] > max_bound, max_bound, df[column])
    df[column] = np.where(df[column] < min_bound, min_bound, df[column])

# Min-Max Scaling using .min() and .max() Pandas methods
def min_max_scaling(df):    
    df_norm = df.copy()
    for column in df_norm.columns:
        df_norm[column] = (df_norm[column] - df_norm[column].min()) / (df_norm[column].max() - df_norm[column].min())        
    return df_norm

In [4]:
# Data Preprocessing

# Drop the columns Delivery Phase(1: 90, 2: 2, NaN: 4) and Education(5: 93, NaN: 3)
df = df.drop(["Delivery phase", "Education", "Community"], axis = 1)

# Replacing Nan of Weights with the Mean of its respective Result category
mean_0 = (df.loc[df['Result'] == 0])['Weight'].mean()
mean_1 = (df.loc[df['Result'] == 1])['Weight'].mean()

df["Weight"] = np.where((df["Result"] == 0) & (df["Weight"].isna()), mean_0, df["Weight"])
df["Weight"] = np.where((df["Result"] == 1) & (df["Weight"].isna()), mean_1, df["Weight"])

# For now, Filling Numeric Columned NaN Values with Mean
df["Age"] = df["Age"].fillna(df["Age"].mean())
df["HB"] = df["HB"].fillna(df["HB"].mean())
df["BP"] = df["BP"].fillna(df["BP"].mean())

# Very Basic Method of taking care of Outliers(Replace with IQR, Min-Max) for Age & BP columns
scale_outlier(df, "Age")
scale_outlier(df, "BP")

# Labelling Residence = 2 as Residence = 0 to get Binary Labelled Column (Before: Residence(1,2), After: Residence(1,0))
df["Residence"] = np.where(df["Residence"] == 2, 0, df["Residence"])
# Filling NaN with Mode = 1
df["Residence"] = df["Residence"].fillna(1)

# Converting IFA(int) to IFA(float)
df["IFA"] = df["IFA"].astype(float)

# Moving converted Float Result, to get it as the last Column
res = df["Result"].astype(float)
df = df.drop(["Result"], axis = 1)
df["Result"] = res

# Performing Normalization of the dataset (into ranges from 0 to 1) using Pandas
df = min_max_scaling(df)

In [14]:
# Creating Train-Test Splits of the dataset using .train_test_split() in Sklearn
from sklearn.model_selection import train_test_split
X = df.iloc[:,:-1].values
y = df.iloc[:,-1:].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [15]:
class layer():
    def __init__(self, input_units, output_units, alpha = 0.00001, activation = 'tanh'):
        self.alpha = alpha
        self.activation = 'tanh'
        self.weights = np.random.normal(loc=0.0, 
                                        scale = np.sqrt(2/(input_units+output_units)), 
                                        size = (input_units,output_units))
        self.bias = np.zeros(output_units)
        
    def forward_prop(self, input):
        if self.activation == 'tanh':
            return np.tanh(np.dot(input, self.weights) + self.bias)
    
    def backward_prop(self):
        pass        

In [18]:
input_units = 6
output_units = 1
classifier = []
classifier.append(layer(input_units, 20))
classifier.append(layer(20, 10))
classifier.append(layer(10, output_units))

In [19]:
inputs = X_train[0]

for layer in classifier:
    inputs = layer.forward_prop(inputs)

    print(inputs)

[ 0.41016657  0.55942498 -0.2467484   0.228636   -0.33030647  0.7408246
 -0.05673701  0.16383175  0.49818132 -0.32100873  0.12102824  0.16597796
  0.61680637 -0.42938831 -0.46270697  0.00206963  0.19063535 -0.67032432
  0.61981794  0.1146678 ]
[-0.42524235  0.114634   -0.15210979 -0.11987357 -0.13368837  0.31617895
 -0.13558228  0.15873677 -0.49031323  0.13423219]
[-0.13778395]


[0.47863248 0.28571429 0.62286733 0.         0.513274   1.        ]
